In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [3]:
#train_val_split=tfds.Split.TRAIN.subsplit([6,4])

(train_data,validation_data),test_data=tfds.load(
    name="imdb_reviews",
    split=(('train[:60%]','train[60%:]'),'test'),
    as_supervised=True)


In [13]:

for i in train_data.take(4):
    print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b'This is a big step down after the surprisingly enjoyable original. This sequel isn\'t nearly as fun as part one, and it instead spends too much time on plot development. Tim Thomerson is still the best thing about this series, but his wisecracking is toned down in this entry. The performances are all adequate, but this time the script lets us down. The action is merely routine and the plot is only mildly interesting, so I need lots of silly laughs in order to stay entertained during a "Trancers" movie. Unfortunately, the laughs are few and far between, and so, this film is watchable at best.'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"Perhaps because I was so young, innocent and BRAINWASHED when I saw it, this movie was the cause of many sleepless nights for me. I haven't seen it since I was in seventh grade at a Presbyterian school, so I am not sure what effect it would have on me now. Howeve

In [7]:
#Expore data
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_labels_batch


<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 1, 0, 1, 0, 1, 1, 1, 0], dtype=int64)>

In [5]:
#Embed text into values
#usinf google news embedding model

embedding="https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer=hub.KerasLayer(embedding,input_shape=[],
                        dtype=tf.string,trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 2.209591  , -2.7093675 ,  3.6802928 , -1.0291991 , -4.1671185 ,
        -2.4566064 , -2.2519937 , -0.36589956,  1.9485804 , -3.1104462 ,
        -2.4610963 ,  1.3139242 , -0.9161584 , -0.16625322, -3.723651  ,
         1.8498232 ,  3.499562  , -1.2373022 , -2.8403084 , -1.213074  ],
       [ 1.9055302 , -4.11395   ,  3.6038654 ,  0.28555924, -4.658998  ,
        -5.5433393 , -3.2735848 ,  1.9235417 ,  3.8461034 ,  1.5882455 ,
        -2.64167   ,  0.76057523, -0.14820506,  0.9115291 , -6.45758   ,
         2.3990374 ,  5.0985413 , -3.2776263 , -3.2652326 , -1.2345369 ],
       [ 3.6510668 , -4.7066135 ,  4.71003   , -1.7002777 , -3.7708545 ,
        -3.709126  , -4.222776  ,  1.946586  ,  6.1182513 , -2.7392752 ,
        -5.4384456 ,  2.7078724 , -2.1263676 , -0.7084146 , -5.893995  ,
         3.1602864 ,  3.8389287 , -3.318196  , -5.1542974 , -2.4051712 ]],
      dtype=float32)>

In [10]:
#model outline

model=tf.keras.Sequential([
    hub_layer,
    tf.keras.layers.Dense(16,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [11]:
#model compiling
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [12]:
history=model.fit(train_data.shuffle(10000).batch(512),epochs=10,validation_data=validation_data.batch(512),verbose=2)

Epoch 1/10
30/30 - 4s - loss: 0.7998 - accuracy: 0.5374 - val_loss: 0.6800 - val_accuracy: 0.5623
Epoch 2/10
30/30 - 2s - loss: 0.6526 - accuracy: 0.6156 - val_loss: 0.6315 - val_accuracy: 0.6574
Epoch 3/10
30/30 - 2s - loss: 0.6066 - accuracy: 0.6801 - val_loss: 0.5894 - val_accuracy: 0.6921
Epoch 4/10
30/30 - 2s - loss: 0.5657 - accuracy: 0.7133 - val_loss: 0.5543 - val_accuracy: 0.7226
Epoch 5/10
30/30 - 2s - loss: 0.5280 - accuracy: 0.7462 - val_loss: 0.5236 - val_accuracy: 0.7470
Epoch 6/10
30/30 - 3s - loss: 0.4928 - accuracy: 0.7705 - val_loss: 0.4936 - val_accuracy: 0.7690
Epoch 7/10
30/30 - 2s - loss: 0.4571 - accuracy: 0.7963 - val_loss: 0.4652 - val_accuracy: 0.7895
Epoch 8/10
30/30 - 2s - loss: 0.4233 - accuracy: 0.8188 - val_loss: 0.4367 - val_accuracy: 0.8065
Epoch 9/10
30/30 - 3s - loss: 0.3904 - accuracy: 0.8361 - val_loss: 0.4147 - val_accuracy: 0.8167
Epoch 10/10
30/30 - 3s - loss: 0.3612 - accuracy: 0.8487 - val_loss: 0.3933 - val_accuracy: 0.8270


In [29]:
#evaluate
results=model.evaluate(test_data.batch(512),verbose=0)
for name,value in zip(model.metrics_names,results):
    print("%s %0.3f"%(name,value))

loss 0.174
accuracy 0.932
